# Converting hierarchical brain atlas from MNI152NLin6ASym to MNI152NLin2009cASym space
This notebook will convert the hierarchical brain atlas into MNI152NLin2009cASym space since it is the space used by fmriprep and qsiprep. This will harmonize all analyses.

In [1]:
import requests
import nibabel as nib
import ants

## Download the necessary files from templateflow
We will download the T1w in 2mm voxel and the transformation files from templateflow.

In [8]:
files = [
    "https://templateflow.s3.amazonaws.com/tpl-MNI152NLin2009cAsym/tpl-MNI152NLin2009cAsym_res-02_desc-brain_T1w.nii.gz",
    "https://templateflow.s3.amazonaws.com/tpl-MNI152NLin2009cAsym/tpl-MNI152NLin2009cAsym_from-MNI152NLin6Asym_mode-image_xfm.h5",
]

for file in files:
    fname = file.split("/")[-1]
    print(f"Downloading {fname}...")
    with open(f"./data/{fname}", "wb") as f:
        response = requests.get(file)
        f.write(response.content)

## Transform the atlas into MNI152NLin2009cASym space

In [6]:
atlases = [
    "./data/A424.nii.gz",
    "./data/A424+2mm.nii.gz",
    "./data/A424+4mm.nii.gz",
    "./data/A424+n90merged.nii.gz",
]

In [ ]:
fixed = ants.image_read( "./data/tpl-MNI152NLin2009cAsym_res-02_desc-brain_T1w.nii.gz" )
transform = "./data/tpl-MNI152NLin2009cAsym_from-MNI152NLin6Asym_mode-image_xfm.h5"

bids_name = "space-MNI152NLin2009cAsym_res-02_dseg.nii.gz"
space = "space-MNI152NLin2009cAsym_res-02"

for atlas in atlases:
    atlas_name = atlas.split("/")[-1].replace(".nii.gz", "")
    # if atlas_name.startswith("A424+"):
    #     atlas_name, desc_suffix = atlas_name.split("+")
    #     desc_suffix = f"_desc-{desc_suffix}"
    # else:
    #     desc_suffix = ""

    # out_name = f"atlas-{atlas_name}_{space}{desc_suffix}_dseg.nii.gz"

    out_name = f"atlas-{atlas_name}_{bids_name}"

    moving = ants.image_read(atlas)
    warped_image = ants.apply_transforms(fixed=fixed, moving=moving, transformlist=transform, interpolator="genericLabel", verbose=True)
    warped_image.image_write(out_name)

['-d', '3', '-i', '0x14bb3a088', '-o', '0x14bb39408', '-r', '0x103106ce8', '-n', 'genericLabel', '-t', './data/tpl-MNI152NLin2009cAsym_from-MNI152NLin6Asym_mode-image_xfm.h5']
Using double precision for computations.
Input scalar image: 0x14bb3a088
Could not create ImageIO for the input file, assuming dimension = 3 and scalar pixel type
Reference image: 0x103106ce8
The composite transform comprises the following transforms (in order): 
  1. ./data/tpl-MNI152NLin2009cAsym_from-MNI152NLin6Asym_mode-image_xfm.h5[0] (type = AffineTransform)
  2. ./data/tpl-MNI152NLin2009cAsym_from-MNI152NLin6Asym_mode-image_xfm.h5[1] (type = DisplacementFieldTransform)
Default pixel value: 0
Interpolation type: LabelImageGenericInterpolateImageFunction
Output warped image: 0x14bb39408
['-d', '3', '-i', '0x14bb39408', '-o', '0x14bb38608', '-r', '0x103106ce8', '-n', 'genericLabel', '-t', './data/tpl-MNI152NLin2009cAsym_from-MNI152NLin6Asym_mode-image_xfm.h5']
Using double precision for computations.
Input sc

## Process metadata into corresponding tsv and json sidecars

In [11]:
import pandas as pd

In [22]:
df = pd.read_csv("./data/A424_Labels_AA-AAc_main_maps.csv",)
df = df[['A424', 'Nodes', 'ROI']]
df.columns = ['index', 'label', 'region_name']

In [23]:
json_sidecar = {
    "Name": "A424",
    "Authors": [
        "Chadi G. Abdallah",
        "Teddy J. Akiki",
    ],
    "BIDSVersion": "1.10.0",
    "Description": "A424 atlas in MNI152NLin2009cAsym space",
}

In [25]:
# bids_name = "space-MNI152NLin2009cAsym_res-02_dseg"


for atlas in atlases:
    atlas_name = atlas.split("/")[-1].replace(".nii.gz", "")
    out_name = f"atlas-{atlas_name}_dseg"

    json_sidecar["Space"] = "MNI152NLin2009cAsym"
    json_sidecar["Resolution"] = "2mm"
    json_sidecar["Name"] = atlas_name

    with open(f"{out_name}.json", "w") as f:
        import json
        json.dump(json_sidecar, f, indent=4)

    df.to_csv(f"{out_name}.tsv", sep="\t", index=False)